## Word Sense Disambiguation

In this exercise I will:
- Do automatic word sense disambiguation (WSD) using the context around an ambigious word
- Apply the semantic knowledge in lexicons and WordNet to this task

## Getting started

This requires that you have downloaded the following NLTK corpora/lexicons:

In [4]:
import nltk
nltk.download('senseval')
nltk.download('wordnet')

[nltk_data] Downloading package senseval to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package senseval is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Accessing relevant modules and packages:

In [2]:
from nltk.corpus import senseval
from nltk.corpus import wordnet as wn
import numpy as np
from collections import defaultdict,Counter
from sklearn.tree import DecisionTreeClassifier
from nltk.wsd import lesk
from sklearn.model_selection import cross_val_score
from nltk.stem import WordNetLemmatizer 
import urllib.request
import urllib.request

This function is for the evaluation of features as we go along.

In [3]:
def decision_stump_eval(L1,L2):
    '''Returns crossvalidated accuracy for a two class classificaion problem using a decision stump 
    based on a single feature. L1 are feature values for first class, L2 feature values for second class'''
    labels = [0]*len(L1) + [1]*len(L2)
    data = np.array(L1 + L2).reshape(-1, 1)
    clf = DecisionTreeClassifier(max_depth=1)
    return np.mean(cross_val_score(clf, data, labels, cv=5))

### Setting up the WSD task

I will be using the Senseval corpus, which has sense-tagged data for a small set of word types. Here, I will only look at the ambiguity of the word *line*. Note that this corpus is arranged in a way that is **NOT** typical for NLTK corpora, and so I have provided some starter code that will allow you to iterate over each "instance", which provides a case of the word *line* in some context, as well as information about which sense it is in that context. You will need to inspect this data manually to figure out how to use it.

Reorganize the dataset into two Python dictionaries, `test_set` and `dev_set`, where the keys of each dictionary are the senses, and the values are a list of instances where *line* has that sense. The `test_set` should contain the first 200 instances per sense, and `dev_set` should contain all other instances for that sense (Each sense does have more than 200 instances). Going forward, I have used `dev_set` exclusively for inspecting output, but I have reported numbers from `test_set`.

In [4]:
dev_set = defaultdict(list)
test_set = defaultdict(list)

for instance in senseval.instances('line.pos'):
    
    sense = instance.senses[0]
    if sense in test_set and len(test_set[sense]) == 200:
        dev_set[sense].append(instance)
    else:
        test_set[sense].append(instance)

In [5]:
assert len(dev_set['product']) == 2017
assert len(test_set['product']) == 200
        
print('Success!')

Success!


### Synset mapping

To carry out a few of the exercises in this exercise, I will first need to associate each sense in the Senseval dataset with a synset in WordNet. First, printing out all the synsets associated with the word *line* along with their gloss (Don't be surprised to find that there are a lot of them, many more than the senses in Senseval!), and creating a Python dictionary which maps each sense to the best corresponding WordNet synset. It might also be useful for you to look at examples of each sense in `dev_set` for this.

In [9]:
for synset in wn.synsets('line'):
    print(synset)
    print(synset.definition())

Synset('line.n.01')
a formation of people or things one beside another
Synset('line.n.02')
a mark that is long relative to its width
Synset('line.n.03')
a formation of people or things one behind another
Synset('line.n.04')
a length (straight or curved) without breadth or thickness; the trace of a moving point
Synset('line.n.05')
text consisting of a row of words written across a page or computer screen
Synset('line.n.06')
a single frequency (or very narrow band) of radiation in a spectrum
Synset('line.n.07')
a fortified position (especially one marking the most forward position of troops)
Synset('argumentation.n.02')
a course of reasoning aimed at demonstrating a truth or falsehood; the methodical process of logical reasoning
Synset('cable.n.02')
a conductor for transmitting electrical or optical signals or electric power
Synset('course.n.02')
a connected series of events or actions or developments
Synset('line.n.11')
a spatial location defined by a real or imaginary unidimensional ex

In [9]:
synset_lookup = {}

synset_lookup['cord'] = wn.synset('line.n.18')
synset_lookup['division'] = wn.synset('line.n.29')
synset_lookup['formation'] = wn.synset('line.n.03')
synset_lookup['product'] = wn.synset('line.n.22')
synset_lookup['text'] = wn.synset('line.n.05')
synset_lookup['phone'] = wn.synset('telephone_line.n.02')


### Lesk

Applying the Lesk algorithm for WSD and printing out the accuracy for the `test_set`. I also indicate the most common sense baseline for this test set.

In [11]:
correct = 0
total = 0

for sense in test_set:
    for instance in test_set[sense]:
        total += 1
        if lesk([pair[0].lower() for pair in instance.context], 'line', 'n') == synset_lookup[sense]:
            correct += 1
            
print('result')           
print(correct/total)
print('most common sense')
print(1/6)

result
0.005
most common sense
0.16666666666666666


In [12]:
for sense in dev_set:
    print(sense)
    for instance in dev_set[sense][:10]:
        print(lesk([pair[0].lower() for pair in instance.context], 'line', 'n'))

cord
Synset('agate_line.n.01')
Synset('production_line.n.01')
Synset('agate_line.n.01')
Synset('wrinkle.n.01')
Synset('wrinkle.n.01')
Synset('occupation.n.01')
Synset('wrinkle.n.01')
Synset('line.n.20')
Synset('agate_line.n.01')
Synset('wrinkle.n.01')
division
Synset('occupation.n.01')
Synset('wrinkle.n.01')
Synset('production_line.n.01')
Synset('agate_line.n.01')
Synset('argumentation.n.02')
Synset('production_line.n.01')
Synset('line.n.14')
Synset('line.n.20')
Synset('production_line.n.01')
Synset('production_line.n.01')
formation
Synset('agate_line.n.01')
Synset('wrinkle.n.01')
Synset('agate_line.n.01')
Synset('lineage.n.01')
Synset('wrinkle.n.01')
Synset('occupation.n.01')
Synset('line.n.20')
Synset('agate_line.n.01')
Synset('line.n.20')
Synset('production_line.n.01')
phone
Synset('wrinkle.n.01')
Synset('line.n.20')
Synset('occupation.n.01')
Synset('agate_line.n.01')
Synset('agate_line.n.01')
Synset('wrinkle.n.01')
Synset('wrinkle.n.01')
Synset('argumentation.n.02')
Synset('occupat

Discussing the results from the LESK algorithm:

The results are well below the most common sense baselines. This is because Lesk actually keeps choosing senses that aren't even among the senses included in the Senseval dataset. An easy way to improve things considerably would be to limit Lesk to the senses actually represented in the `test_set`. However, this won't solve the deep problem that the words in the context don't seem to be appearing in the glosses.

#### Re-implementing the Lesk algorthm based on the flaws I saw above 

In [13]:
def my_lesk(context_sentence,ambiguous_word):
    '''does the lesk algorithm but limits the output to the synsets in synset_lookup'''
    context = set(context_sentence)
    synsets = wn.synsets(ambiguous_word)    
    return max((len(context.intersection(ss.definition().split())), ss) for ss in synsets if ss in synset_lookup.values())[1]

correct = 0
total = 0
for sense in test_set:
    for instance in test_set[sense]:
        total += 1
        if my_lesk([pair[0].lower() for pair in instance.context], 'line') == synset_lookup[sense]:
            correct += 1
            
print("result")           
print(correct/total)


result
0.22833333333333333


### WordNet distance

Trying to do WSD by picking the synset that is closest (in WordNet) to the synsets of mostly non-ambigious context words.


The biggest challenge in this problem is identifying "mostly" non-ambiguous words. I could exclude any word type that has any polysemy (i.e. associated with more than one synset), but that seems too extreme. Instead, I am going to consider a word mostly non-ambiguous if it appears as one particular sense 75% of the time, based on the corpus counts provided in WordNet. I have written  a general function, `get_dominant_sense`, which takes a word and a POS (a single letter, same as the input to the WordNet lemmatizer), and returns the dominant (75% of instances) synset if it exists, or `None` if it doesn't. The POS will be useful because, in order to do this properly, I will have to correctly lemmatize the word, so as to match it with the lemmas of each of its synset, so you can get the right count.

In [6]:
dominant_sense_ratio = 0.75
lemmatizer = WordNetLemmatizer()

def get_dominant_sense(word,pos):
    if pos.startswith("N"):
        lemma = lemmatizer.lemmatize(word,'n')
    elif pos.startswith("V"):
        lemma = lemmatizer.lemmatize(word,'v')
    else:
        lemma = word
    syn_counts = []
    total = 0
    for synset in wn.synsets(word):
        for wn_lemma in synset.lemmas():
            if lemma == wn_lemma.name():
                lemma_count = wn_lemma.count()
                total += lemma_count
                syn_counts.append([lemma_count,synset])
    syn_counts.sort()
    if total > 0 and syn_counts[-1][0]/total >= dominant_sense_ratio:
        return syn_counts[-1][1]
    else:
        return None


In [7]:
assert str(get_dominant_sense("word","n")) == "Synset('word.n.01')"
assert get_dominant_sense("fun","n") is None
print("Success!")

Success!


#### 3.2

rubric={accuracy:2}

Now write a `get_wordnet_context_similarities` function which builds a list of synsets associated with nouns in the context for an given instance (using the `get_dominant_sense` function from **3.1**) and then calculates the average Wu-Palmer similarity from each of those synsets to the synsets associated with each of the possible senses of line ( and the `synset lookup` from **1**). If you correctly implement `get_wordnet_context_similarities`, the code provided below will calculate the WSD accuracy on the `test_set` (don't be surprised if it is fairly slow, and the result isn't that good).

In [11]:
def get_wordnet_context_similarities(instance,synset_lookup):
    '''returns a dictionary of average Wu-Palmer similarities between the synsets in synset lookup 
    to the dominent senses of the nouns in the context of the instance'''
    context_similarity_dict = defaultdict(int)
    # your code here
    total = 0
    for word_pos_pair in instance.context:
        if len(word_pos_pair) == 2 and word_pos_pair[1].startswith("N"):
            token,pos = word_pos_pair
            context_synset = get_dominant_sense(token,pos)
            if not context_synset is None:
                for word_sense,word_synset in synset_lookup.items():
                    similarity = synset_lookup[word_sense].wup_similarity(context_synset)
                    if not similarity is None:
                        total +=1
                        context_similarity_dict[word_sense] += similarity
    for sense in context_similarity_dict:
        context_similarity_dict[sense] /= total
    # your code here
    return context_similarity_dict

def get_closest_sense(instance,synset_lookup):
    '''gets the sense of the senses in synset_look that is on average most similar to
    the words in the context of instance'''
    context_similarities = get_wordnet_context_similarities(instance,synset_lookup)
    if len(context_similarities) == 0:
        return None
    
    max_value = max(context_similarities.values())
    for sense,value in context_similarities.items():
        if value == max_value:
            return sense 

correct = 0
total = 0
for sense in test_set:
    for instance in test_set[sense]:
        total += 1
        if get_closest_sense(instance,synset_lookup) == sense:
            correct += 1
            
print(correct/total)

0.18666666666666668


### Exercise 4: Hand-crafted lexicon

rubric={accuracy:2}

Pick a sense of "line", and examine some of the contexts for instances of that sense in the `dev_set`. Based on this, build a small lexicon of words (at least 5) that consistently appear in contexts for that sense, and, using the provided `decision_stump_eval` function, show that this small lexicon works as a useful feature for distinguishing that sense from all other senses (you just need to pass `decision_stump_eval` two lists of numbers corresponding to the counts for the instances of the two different senses). A simple function for counting the number of times a word from a lexicon appears in a semeval instance is provided (note that there is an error in the semeval annotation which it handles!)

In [32]:
def count_occurrences_in_context(instance,lexicon):
    '''count how often the words in lexicon appear in the context of the provided semeval instance'''
    count = 0
    for pair in instance.context:
        if len(pair) == 2:
            word,pos = pair
            if word in phone_words:
                count += 1
    return count

#your code here
phone_words = set(["telephone","phone","dial",'telecommunications',"block"])

def count_all(instances,lexicon):
    return [count_occurrences_in_context(instance, lexicon) for instance in instances]

counts_for_phone_sense = count_all(test_set["phone"],phone_words)

for other_sense in test_set:
    if other_sense != "phone":
        counts_for_other_sense = count_all(test_set[other_sense],phone_words)
        print(other_sense)
        print(decision_stump_eval(counts_for_other_sense,counts_for_phone_sense))
#your code here

cord
0.8174999999999999
division
0.8175000000000001
formation
0.8175000000000001
product
0.8125
text
0.82


### Exercise 5: Concreteness

rubric={accuracy:2,quality:1}

One typical distinction between senses of a word is that some senses are more concrete (involving the physical world) whereas others are more abstract. A list of words with human-assigned concreteness ratings can be found on the webpage [here](https://raw.githubusercontent.com/ArtsEngine/concreteness/master/Concreteness_ratings_Brysbaert_et_al_BRM.txt). The code below will extract it into a Python dictionary for your use.

In [19]:
#provided code
concreteness_dict = {}

for line in urllib.request.urlopen("https://raw.githubusercontent.com/ArtsEngine/concreteness/master/Concreteness_ratings_Brysbaert_et_al_BRM.txt").read().decode("utf-8").split("\n"):
    if line.startswith("Word") or not line.strip():
        continue
    line = line.split("\t")
    concreteness_dict[line[0]] = float(line[2])
    
print(len(concreteness_dict))

39954


Choose two senses of *line* that you think should be distinguishable based on concreteness. Calculate an average concreteness for the contexts of each instance of each sense, and print the results of using `decision_stump_eval` on those concreteness values (separated by sense) as well the total average context concreteness for each sense. Together, these should show that your hypothesis was correct. If it wasn't, try a different pair of senses. 

In [20]:
def calculate_concreteness(instance,concreteness_dict):
    '''calcualte average concreteness based on the words of the context of instance and the
    provided concreteness_diction for each word'''
    concreteness = 0
    total_words = 0
    for pair in instance.context:
        if len(pair) == 2:
            word = pair[0].lower() 
            if word in concreteness_dict:
                concreteness += concreteness_dict[word]
                total_words += 1
    return concreteness/total_words

def get_concretenesses_for_instances(instances,concreteness_dict):
    L = []
    for instance in instances:
        L.append(calculate_concreteness(instance,concreteness_dict))
    return L

L1 = get_concretenesses_for_instances(test_set["cord"],concreteness_dict)
L2 = get_concretenesses_for_instances(test_set["division"],concreteness_dict)
print("cord concreteness:")
print(np.mean(L1))
print("division concreteness:")
print(np.mean(L2))
print("decision stump evaluation:")
print(decision_stump_eval(L1,L2))

cord concreteness:
2.7302934692625183
division concreteness:
2.4681902854215854
decision stump evaluation:
0.7575000000000001


### Exercise 6: Hypernyms

#### 6.1

rubric={accuracy:2, efficiency:1}

In this exercise you'll be looking at the WordNet hypernyms of words appearing in the context as potential features for doing WSD. First, you need to write a recursive function `get_all_hypernyms` which collects the names (e.g. `synset.name()`) of a provided WordNet synset and all of its hypernyms.

In [33]:
def get_all_hypernyms(synset):
    '''return a list of all the WordNet hypernyms of a provided synset'''
    # your code here
    hypernyms = [synset.name()]
    for hypernym in synset.hypernyms():
        if not hypernym is None:
            hypernyms.extend(get_all_hypernyms(hypernym))
    return hypernyms
    # your code here

In [34]:
assert get_all_hypernyms(wn.synsets("cat")[0]) == ['cat.n.01', 'feline.n.01', 'carnivore.n.01', 'placental.n.01', 'mammal.n.01', 'vertebrate.n.01', 'chordate.n.01', 'animal.n.01', 'organism.n.01', 'living_thing.n.01', 'whole.n.02', 'object.n.01', 'physical_entity.n.01', 'entity.n.01']
print("Success!")

Success!


#### 6.2 (Optional)

rubric={accuracy:1}

Next, use the `get_dominant_sense` function from **2.1** and the `get_all_hypernyms` function from **6.1** to create, for each instance in your dataset, a Counter of all the hypernyms for all  words (which have a single dominant sense) in the context. You should use this to create a Python dictionary analogous to `test_set`, except the values are a list of Counters with the counts of context synsets (names). At the same time, you should maintain a separate Counter, `total_counter` which counts the appearance of these synsets across the entire dataset

In [37]:
total_counter = Counter()

#your code here

def get_synsets_for_instance(instance):
    '''Get a counter of all the hypernym synset in instance'''
    counts = Counter()
    for word_pos_pair in instance.context:
        if len(word_pos_pair) == 2:
            token,pos = word_pos_pair
            synset = get_dominant_sense(token,pos)
            if not synset is None:
                counts.update(get_all_hypernyms(synset))
    return counts

instance_counts_dict = defaultdict(list)
for sense in test_set:
    for instance in test_set[sense]:
        instance_counts = get_synsets_for_instance(instance)
        instance_counts_dict[sense].append(instance_counts)
        total_counter.update(instance_counts)

#your code here


In [38]:
assert len(total_counter) == 3971
assert total_counter.most_common(1)[0] == ('entity.n.01',6615)
print("Success!")

Success!


#### 6.3 (Optional)

rubric={accuracy:1}

Print out the 100 most common synsets across the entire dataset. Among this top 100, pick a synset that you think will distinguish two of the senses. Briefly justify your choice in the markdown box below. Then test whether it indeed does. You'll want to normalize your counts of the hypernyms in some way, e.g. using the total number of hypernyms for that instance. You should show (i) whether the synset from the context does distinguish the two contexts using `decision_stump_eval` and (ii) whether the directionality is what you would expect. It is okay if your hypothesis is wrong, provided your original idea was sensible.

In [39]:
print(total_counter.most_common(100))

[('entity.n.01', 6615), ('physical_entity.n.01', 4032), ('abstraction.n.06', 2581), ('object.n.01', 2509), ('whole.n.02', 2310), ('artifact.n.01', 1139), ('living_thing.n.01', 1138), ('organism.n.01', 1135), ('causal_agent.n.01', 1085), ('in.r.01', 1047), ('person.n.01', 1044), ('psychological_feature.n.01', 707), ('instrumentality.n.03', 602), ('measure.n.02', 571), ('group.n.01', 551), ('event.n.01', 531), ('state.v.01', 461), ('express.v.02', 461), ('act.n.02', 377), ('along.r.01', 374), ('social_group.n.01', 359), ('attribute.n.02', 319), ('equally.r.01', 302), ('fundamental_quantity.n.01', 290), ('time_period.n.01', 280), ('organization.n.01', 277), ('matter.n.03', 271), ('device.n.01', 262), ('communication.n.02', 245), ('by.r.01', 244), ('merely.r.01', 225), ('structure.n.01', 220), ('new.a.01', 213), ('one.s.01', 201), ('relation.n.01', 188), ('definite_quantity.n.01', 175), ('cognition.n.01', 171), ('act.v.01', 161), ('machine.n.01', 161), ('conveyance.n.03', 150), ('activity.

Answer:

I choose the object synset to distinguish the "cord" and "division" senses of *line*. My expectation is that use of line in the sense of "cord" will often involve physical objects, where as "division" is more abstract and won't involve physical objects as much.

In [40]:
def get_normalized_synset_counts(list_of_counts, synset):
    output = []
    for counts in list_of_counts:
        total = sum(counts.values())
        if total == 0 or synset not in counts:
            output.append(0)
        else:
            output.append(counts[synset]/total)
    return output

def check_if_synset_distinguishes_sense(synset,sense1,sense2,verbose=True):
    sense1_counts = get_normalized_synset_counts(instance_counts_dict[sense1],synset)
    if verbose:
        print("Normalized count for ",  sense1)
        print(np.mean(sense1_counts))
    sense2_counts = get_normalized_synset_counts(instance_counts_dict[sense2],synset)
    if verbose:
        print("Normalized count for ",  sense2)
        print(np.mean(sense2_counts))
    return decision_stump_eval(sense1_counts,sense2_counts)
print("Eval")
print(check_if_synset_distinguishes_sense("object.n.01","cord","division"))

Eval
Normalized count for  cord
0.03949330292738305
Normalized count for  division
0.027364713915378668
0.59
